In [19]:
from data import get_dataset

import time
import utils
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
from early_stop import EarlyStopping, Stop_args
from model import TransformerModel
from lr import PolynomialDecayLR
import os.path
import torch.utils.data as Data
import argparse
import networkx as nx
import pandas as pd
import dgl

In [47]:
import utils
import dgl
import torch
from ogb.nodeproppred import DglNodePropPredDataset
import scipy.sparse as sp
import os.path
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from dgl.data import  CoraFullDataset, AmazonCoBuyComputerDataset, AmazonCoBuyPhotoDataset,CoauthorCSDataset,CoauthorPhysicsDataset
import random
# from cache_sample import cache_sample_rand_csr

def get_dataset(dataset, pe_dim):
    if dataset in {"ogbn-arxiv", "ogbn-products", "ogbn-proteins", "ogbn-papers100M", "ogbn-mag"}:
        if dataset == "ogbn-arxiv":
            dataset = DglNodePropPredDataset(name="ogbn-arxiv")
        elif dataset == "ogbn-products":
            dataset = DglNodePropPredDataset(name="ogbn-products")
        elif dataset == "ogbn-proteins":
            dataset = DglNodePropPredDataset(name="ogbn-proteins")
        elif dataset == "ogbn-papers100M":
            dataset = DglNodePropPredDataset(name="ogbn-papers100M")
        elif dataset == "ogbn-mag":
            dataset = DglNodePropPredDataset(name="ogbn-mag")
        split_idx = dataset.get_idx_split()
        graph, labels = dataset[0]
        features = graph.ndata['feat']
        adj = graph.adj(scipy_fmt="csr")
        # adj = cache_sample_rand_csr(adj, s_len)
        # print(labels)

        idx_train = split_idx['train']
        idx_val = split_idx['valid']
        idx_test = split_idx['test']

        graph = dgl.from_scipy(adj)
        lpe = utils.laplacian_positional_encoding(graph, pe_dim) 
        features = torch.cat((features, lpe), dim=1)
        adj = utils.sparse_mx_to_torch_sparse_tensor(adj)
        # print(adj)
        print(labels)
        # labels = torch.argmax(labels, 1)
        labels = labels.reshape(-1)
        print(labels)

    elif dataset in {"pubmed", "corafull", "computer", "photo", "cs", "physics","cora", "citeseer"}:



        file_path = "dataset/"+dataset+".pt"

        data_list = torch.load(file_path)

        # data_list = [adj, features, labels, idx_train, idx_val, idx_test]
        adj = data_list[0]
        print(adj)
        features = data_list[1]
        labels = data_list[2]

        idx_train = data_list[3]
        idx_val = data_list[4]
        idx_test = data_list[5]

        if dataset == "pubmed":
            graph = PubmedGraphDataset()[0]
        elif dataset == "corafull":
            graph = CoraFullDataset()[0]
        elif dataset == "computer":
            graph = AmazonCoBuyComputerDataset()[0]
        elif dataset == "photo":
            graph = AmazonCoBuyPhotoDataset()[0]
        elif dataset == "cs":
            graph = CoauthorCSDataset()[0]
        elif dataset == "physics":
            graph = CoauthorPhysicsDataset()[0]
        elif dataset == "cora":
            graph = CoraGraphDataset()[0]
        elif dataset == "citeseer":
            graph = CiteseerGraphDataset()[0]

        graph = dgl.to_bidirected(graph)
        # graph = dgl.graph(graph)
        return adj, graph
        # lpe = utils.laplacian_positional_encoding(graph, pe_dim) 
        # lpe = utils.randomwalk_positional_encoding(graph)
        
        features = torch.cat((features, lpe), dim=1)



    elif dataset in {"aminer", "reddit", "Amazon2M"}:

 
        file_path = './dataset/'+dataset+'.pt'

        data_list = torch.load(file_path)

        #adj, features, labels, idx_train, idx_val, idx_test

        adj = data_list[0]
        
        #print(type(adj))
        features = torch.tensor(data_list[1], dtype=torch.float32)
        labels = torch.tensor(data_list[2])
        idx_train = torch.tensor(data_list[3])
        idx_val = torch.tensor(data_list[4])
        idx_test = torch.tensor(data_list[5])

        graph = dgl.from_scipy(adj)


        lpe = utils.laplacian_positional_encoding(graph, pe_dim) 
       
        features = torch.cat((features, lpe), dim=1)

        adj = utils.sparse_mx_to_torch_sparse_tensor(adj)
        

        labels = torch.argmax(labels, -1)
        

    return adj, features, labels, idx_train, idx_val, idx_test

In [48]:
adj, graph = get_dataset('cora', 3)

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
       values=tensor([0.3333, 0.2887, 0.3333,  ..., 0.0870, 0.2500, 0.2500]),
       size=(2708, 2708), nnz=10556, layout=torch.sparse_coo)
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [50]:
edge_index = adj._indices()
    G = dgl.graph((edge_index[0], edge_index[1]))

RuntimeError: Cannot get indices on an uncoalesced tensor, please call .coalesce() first

In [44]:
dgl.graph(A)

DGLError: dgl.graph no longer supports graph construction from a SciPy sparse matrix, use dgl.from_scipy instead.

In [21]:
adj, features, labels, idx_train, idx_val, idx_test = get_dataset('cora', 3)

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
       values=tensor([0.3333, 0.2887, 0.3333,  ..., 0.0870, 0.2500, 0.2500]),
       size=(2708, 2708), nnz=10556, layout=torch.sparse_coo)
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [11]:
edge_index = adj._indices()
edge_index[0]
g = dgl.graph((edge_index[0], edge_index[1]))

[874,
 1445,
 2606,
 2008,
 1853,
 1986,
 2570,
 607,
 930,
 2358,
 622,
 2125,
 1860,
 30,
 433,
 952,
 1371,
 561,
 2625,
 48,
 1567,
 1937,
 1938,
 2276,
 2565,
 338,
 914,
 1472,
 143,
 477,
 1463,
 1019,
 1691,
 570,
 0,
 2162,
 2438,
 788,
 1272,
 1017,
 981,
 1939,
 2607,
 681,
 869,
 1977,
 401,
 1016,
 1126,
 575,
 1741,
 1432,
 2320,
 2244,
 1192,
 1424,
 2245,
 313,
 2543,
 811,
 2605,
 916,
 795,
 2296,
 2574,
 2349,
 2495,
 182,
 1603,
 351,
 1708,
 327,
 1270,
 1933,
 1119,
 374,
 1364,
 1940,
 590,
 1673,
 1159,
 1636,
 386,
 482,
 2243,
 2247,
 546,
 436,
 1558,
 2696,
 1057,
 2242,
 1496,
 1000,
 205,
 2412,
 982,
 285,
 1807,
 230,
 1439,
 1811,
 2529,
 673,
 267,
 2555,
 2212,
 398,
 1645,
 2668,
 171,
 579,
 1548,
 938,
 229,
 923,
 1929,
 967,
 598,
 2213,
 1321,
 172,
 1861,
 932,
 1403,
 1099,
 1820,
 2669,
 1928,
 2659,
 792,
 1400,
 2425,
 1870,
 1024,
 2038,
 1608,
 2065,
 1107,
 1868,
 2629,
 1434,
 1814,
 892,
 710,
 382,
 1066,
 2108,
 2078,
 411,
 1950,
 1